In [3]:
from keras.models import load_model
import numpy as np
from keras.layers import concatenate, Input
import tensorflow as tf
from keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [4]:
# Load the saved models
alexnet = load_model('../models/alexnet_0_ultrasound')
fcn = load_model('../models/fcn_0_ultrasound')
lenet=load_model('/notebooks/ensemble_nns_major/models/lenet5_0_ultrasound')

In [5]:
# Define an input shape
input_shape = (227,227, 1)

# Create an ensemble model
input_layer = Input(shape=input_shape)
output1 = alexnet(input_layer)
output2 = fcn(input_layer)
output3 = lenet(input_layer)
ensemble_output = concatenate([output1, output2, output3])
ensemble_model = Model(inputs=input_layer, outputs=ensemble_output)


In [6]:
ensemble_model.compile(loss="BinaryCrossentropy", optimizer='Adam', metrics=["BinaryAccuracy"])

In [7]:
image_shape=(227,227)
train_dir="/notebooks/ultrasound/train"
validation_dir=train_dir
test_dir="/notebooks/ultrasound/val"
batch_size=64
test_train_split=0.3

train_data = image_dataset_from_directory(\
      train_dir,color_mode="grayscale",image_size=image_shape ,\
      subset='training',seed=50, validation_split=test_train_split,\
      batch_size=batch_size)
validation_data = image_dataset_from_directory(validation_dir,
      color_mode="grayscale",image_size=image_shape, subset='validation',seed=50,\
      validation_split=test_train_split,batch_size=batch_size)
test_data = image_dataset_from_directory(test_dir,
      color_mode="grayscale",image_size=image_shape, batch_size=batch_size)

Found 8116 files belonging to 2 classes.
Using 5682 files for training.
Found 8116 files belonging to 2 classes.
Using 2434 files for validation.
Found 900 files belonging to 2 classes.


In [8]:
fcn_predict=fcn.predict(test_data,verbose=0).reshape((900,))
alexnet_predict=alexnet.predict(test_data,verbose=0).reshape((900,))
lenet_predict=lenet.predict(test_data,verbose=0).reshape((900,))

In [9]:
op=np.array([])
for i,j in test_data:
    op=np.append(op,j.numpy())

In [10]:
op.shape

(900,)

In [11]:
fcn_predict.shape

(900,)

In [12]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 227, 227, 1  0           []                               
                                )]                                                                
                                                                                                  
 Alexnet (Sequential)           (None, 1)            91716245    ['input_1[0][0]']                
                                                                                                  
 fcn (Sequential)               (None, 1)            683145      ['input_1[0][0]']                
                                                                                                  
 lenet (Sequential)             (None, 1)            5406221     ['input_1[0][0]']            

In [13]:
ensemble_predict = np.max(np.around(ensemble_model.predict(test_data, verbose=0)), axis=1).reshape((900,))

In [14]:
print(ensemble_predict[:10].round())
print(fcn_predict[:10].round())
print(alexnet_predict[:10].round())
print(lenet_predict[:10].round())
print(op[:10])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 1. 1.]
[1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 0. 0. 1. 1. 1. 1. 0. 0.]
[0. 1. 0. 1. 1. 1. 0. 1. 0. 0.]


In [15]:
results = alexnet.evaluate(test_data)
print("test loss, test acc:", results)
results = lenet.evaluate(test_data)
print("test loss, test acc:", results)
results = fcn.evaluate(test_data)
print("FCN test loss, test acc:", results)

15/15 [==============================] - 0s 16ms/step - loss: 1.0917 - binary_accuracy: 0.7911
test loss, test acc: [1.0917024612426758, 0.7911111116409302]
15/15 [==============================] - 0s 12ms/step - loss: 2.3377 - binary_accuracy: 0.8011
test loss, test acc: [2.337690591812134, 0.801111102104187]
15/15 [==============================] - 1s 17ms/step - loss: 0.3499 - binary_accuracy: 0.8833
FCN test loss, test acc: [0.34987306594848633, 0.8833333253860474]


In [16]:
results = ensemble_model.evaluate(test_data)
print("test loss, test acc:", results)

15/15 [==============================] - 1s 31ms/step - loss: 0.9488 - binary_accuracy: 0.8252
test loss, test acc: [0.9488376975059509, 0.8251851797103882]


In [17]:
ensemble_predictions = np.round((fcn_predict + alexnet_predict+lenet_predict) / 3)

# Evaluate accuracy of ensemble predictions
y_test = op# load your test labels here
# ensemble_accuracy = model1.evaluate(X_test, y_test, verbose=0)[1]
# print("Ensemble accuracy: {:.2f}%".format(ensemble_accuracy * 100))

In [18]:
def calculate_metrics(y_true, y_pred):
    """
    Calculate accuracy, sensitivity, and specificity.
    """
    # Define TP, FP, TN, and FN
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calculate metrics
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    
    return accuracy, sensitivity, specificity

In [19]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, alexnet_predict)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.36666666666666664
Sensitivity: 1.0
Specificity: 0.0


In [20]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, lenet_predict)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.3333333333333333
Sensitivity: 1.0
Specificity: 0.0


In [21]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, fcn_predict)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: nan
Sensitivity: nan
Specificity: nan


/tmp/ipykernel_2699/2553318004.py:12: RuntimeWarning: invalid value encountered in long_scalars
  accuracy = (TP + TN) / (TP + FP + TN + FN)
/tmp/ipykernel_2699/2553318004.py:13: RuntimeWarning: invalid value encountered in long_scalars
  sensitivity = TP / (TP + FN)
/tmp/ipykernel_2699/2553318004.py:14: RuntimeWarning: invalid value encountered in long_scalars
  specificity = TN / (TN + FP)


In [22]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, ensemble_predictions)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.45222222222222225
Sensitivity: 0.39
Specificity: 0.502


In [24]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, ensemble_predictions)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.45222222222222225
Sensitivity: 0.39
Specificity: 0.502
